# NX-492: Neuro-X project II
---

## Week 1

### To-do list:
- Load both models
- List joint & muscle/actuator names so you can pick the right ones
- Run a moment-arm vs angle sweep in each engine
- Plot + print quick metrics (RMSE, correlation)
- Note runtime (first “compute vs accuracy” proxy)

#### 1. Imports + config

In [6]:
# Import packages
import os, time
import numpy as np
import matplotlib.pyplot as plt

# Import backends
import mujoco as mj
import opensim as osim

# Import models
OPENSIM_MODEL = "../models/OpenSim/Arm26/arm26.osim"
MUJOCO_XML    = "../models/MyoSim/elbow/myoelbow_2dof6muscles.xml"

In [7]:
# Angle sweep (degrees) used for BOTH engines so they’re comparable
ANGLE_DEG = (-30, 120)
N_STEPS   = 31

np.set_printoptions(suppress=True, linewidth=160)

#### 2. Load models

In [ ]:
# --- OpenSim ---
osim_model = None
if (osim is not None) and os.path.exists(OPENSIM_MODEL):
    osim_model = osim.Model(OPENSIM_MODEL)
    osim_state = osim_model.initSystem()
    print("[OpenSim] Loaded:", OPENSIM_MODEL)
    print("  # muscles:", osim_model.getMuscles().getSize())
    print("  # coords :", osim_model.getCoordinateSet().getSize())
else:
    print("[OpenSim] Not loaded. Check path and Python bindings.")

# --- MuJoCo ---
mj_model = None
mj_data  = None
if (mj is not None) and os.path.exists(MUJOCO_XML):
    mj_model = mj.MjModel.from_xml_path(MUJOCO_XML)
    mj_data  = mj.MjData(mj_model)
    print("[MuJoCo] Loaded:", MUJOCO_XML)
    print("  nq, nv, nu:", mj_model.nq, mj_model.nv, mj_model.nu)
else:
    print("[MuJoCo] Not loaded. Check path and installation.")

#### 3. Peek names to pick the right joint/muscle/actuator

In [ ]:
def list_opensim_names(model):
    coords = [model.getCoordinateSet().get(i).getName() for i in range(model.getCoordinateSet().getSize())]
    muscles = [model.getMuscles().get(i).getName() for i in range(model.getMuscles().getSize())]
    return coords, muscles

def list_mujoco_names(m):
    joints = [mj.mj_id2name(m, mj.mjtObj.mjOBJ_JOINT, j) for j in range(m.njnt)]
    acts   = [mj.mj_id2name(m, mj.mjtObj.mjOBJ_ACTUATOR, a) for a in range(m.nu)]
    return joints, acts

if osim_model:
    coords, muscles = list_opensim_names(osim_model)
    print("[OpenSim] Coordinates:", coords)
    print("[OpenSim] Muscles    :", muscles)

if mj_model:
    joints, acts = list_mujoco_names(mj_model)
    print("[MuJoCo] Joints      :", joints)
    print("[MuJoCo] Actuators   :", acts)

#### 4. Set the pair you want to compare (edit these 2 strings)

In [ ]:
# Example guesses — replace by what you saw printed above:
OSIM_COORD_NAME   = "elbow_flexion"     # OpenSim coordinate (Arm26)
OSIM_MUSCLE_NAME  = "BIClong"           # OpenSim muscle name (example)

MJ_JOINT_NAME     = "elbow_flex_r"      # MuJoCo joint
MJ_ACTUATOR_NAME  = "biceps_long_r"     # MuJoCo actuator

#### 5. Utilities (rmse/corr + degree↔radian helpers)

In [ ]:
def rmse(a,b): return float(np.sqrt(np.mean((a-b)**2)))
def pearson(a,b):
    a = a - np.mean(a); b = b - np.mean(b)
    den = np.linalg.norm(a)*np.linalg.norm(b)
    return 0.0 if den == 0 else float(np.dot(a,b)/den)

def d2r(deg): return deg*np.pi/180.0
def r2d(rad): return rad*180.0/np.pi

#### 6. Moment arm sweep — OpenSim

In [ ]:
def opensim_moment_arm(model, state, coord_name, muscle_name, angle_deg=(-30,120), steps=31):
    coord  = model.getCoordinateSet().get(coord_name)
    muscle = model.getMuscles().get(muscle_name)
    thetas = np.linspace(d2r(angle_deg[0]), d2r(angle_deg[1]), steps)
    r = []
    for th in thetas:
        coord.setValue(state, th)
        model.realizePosition(state)
        r.append(muscle.computeMomentArm(state, coord))
    return r2d(thetas), np.array(r, dtype=float)

osim_t, osim_r = (np.array([]), np.array([]))
t0 = time.perf_counter()
if osim_model:
    osim_t, osim_r = opensim_moment_arm(osim_model, osim_state, OSIM_COORD_NAME, OSIM_MUSCLE_NAME,
                                        angle_deg=ANGLE_DEG, steps=N_STEPS)
t_osim = time.perf_counter() - t0
print(f"[OpenSim] Moment-arm sweep time: {t_osim:.3f}s")

#### 7. Moment arm sweep — MuJoCo

In [ ]:
def mj_joint_id(m, name):
    try: return mj.mj_name2id(m, mj.mjtObj.mjOBJ_JOINT, name)
    except: return -1

def mj_act_id(m, name):
    try: return mj.mj_name2id(m, mj.mjtObj.mjOBJ_ACTUATOR, name)
    except: return -1

def mujoco_moment_arm_numeric(m, d, joint_name, actuator_name, angle_deg=(-30,120), steps=31, eps=1e-6):
    j_id = mj_joint_id(m, joint_name)
    a_id = mj_act_id(m, actuator_name)
    assert j_id >= 0 and a_id >= 0, "Bad joint/actuator name for MuJoCo."

    qpos_i = m.jnt_qposadr[j_id]
    thetas = np.linspace(d2r(angle_deg[0]), d2r(angle_deg[1]), steps)
    r = []
    d.qpos[:] = 0.0; d.qvel[:] = 0.0; d.ctrl[:] = 0.0
    mj.mj_forward(m, d)

    for th in thetas:
        d.qpos[:] = 0.0
        d.qpos[qpos_i] = th
        mj.mj_forward(m, d)

        # central difference of actuator_length w.r.t joint angle
        d.qpos[qpos_i] = th + eps; mj.mj_forward(m, d); Lp = d.actuator_length[a_id]
        d.qpos[qpos_i] = th - eps; mj.mj_forward(m, d); Lm = d.actuator_length[a_id]
        d.qpos[qpos_i] = th;       mj.mj_forward(m, d)
        r.append((Lp - Lm) / (2*eps))
    return r2d(thetas), np.array(r, dtype=float)

mj_t, mj_r = (np.array([]), np.array([]))
t0 = time.perf_counter()
if mj_model:
    mj_t, mj_r = mujoco_moment_arm_numeric(mj_model, mj_data, MJ_JOINT_NAME, MJ_ACTUATOR_NAME,
                                           angle_deg=ANGLE_DEG, steps=N_STEPS)
t_mj = time.perf_counter() - t0
print(f"[MuJoCo] Moment-arm sweep time: {t_mj:.3f}s")

#### 8. Plot + quick metrics

In [ ]:
if osim_t.size and mj_t.size:
    # If grids match, compare directly; else interpolate one onto the other
    if np.allclose(osim_t, mj_t):
        tt = osim_t
        a  = osim_r
        b  = mj_r
    else:
        tt = mj_t
        a  = np.interp(tt, osim_t, osim_r)
        b  = mj_r

    print("RMSE (moment arm):", rmse(a, b))
    print("Pearson r        :", pearson(a, b))
    print("Runtime (s)      : OpenSim =", round(t_osim,3), " | MuJoCo =", round(t_mj,3))

    plt.figure(figsize=(6,4))
    plt.plot(osim_t, osim_r, label="OpenSim")
    plt.plot(mj_t,   mj_r,   label="MuJoCo/MyoElbow", linestyle="--")
    plt.xlabel("Joint angle (deg)")
    plt.ylabel("Moment arm (m)")
    plt.title(f"Moment arm vs angle\n{OSIM_MUSCLE_NAME} / {MJ_ACTUATOR_NAME}")
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("Run both sweeps first (and check names).")